<a href="https://colab.research.google.com/github/TongLuo2021/HPO-DistilBert-With-Ray-and-Optuna/blob/main/HPO_DistilBert_with_Ray_and_Optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Hyperparameter Search with Transformers and Ray Tune, and Optuna
* by Tong Luo

Hyperparameters (HPs) can have significant impact on the final model parameters and model performance. [Study show](https://towardsdatascience.com/hyperparameter-optimization-for-optimum-transformer-models-b95a32b70949) that the most significant HPs are learning rate and training epochs, while other parameters also matters. Depends on the applications such as CV, NLP, the HPs can be significantly.

----------
###TLuo's works are:
1. Ray and Optuna has [default HPs]((https://github.com/huggingface/transformers/blob/main/src/transformers/trainer_utils.py): 
* learning_rate, 
* num_train_epochs, 
* seed, 
* per_device_train_batch_size. 
2. Besides that, I used customized HPO, by defining hp_space_ray function following the [source code](https://github.com/huggingface/transformers/blob/main/src/transformers/trainer_utils.py), and [sgugers's example](https://github.com/huggingface/transformers/blob/main/src/transformers/trainer_utils.py)
3. Uses same frame work for Optuna.
4. Use same frame work, we can tune Other hyper parameters are listed [here](https://simpletransformers.ai/docs/usage/#configuring-a-simple-transformers-model) .
5. For small HPO setting (n_runtime = 3), the total time is similiar. Ray suppose to be faster because it can parallel moltiple tasks. More experiment need to be done here. 
----------
###[References]
* [Hyperparameter Search with Transformers and Ray Tune](https://huggingface.co/blog/ray-tune)
* [Hyperparameter optimization for optimun transformer models](https://towardsdatascience.com/hyperparameter-optimization-for-optimum-transformer-models-b95a32b70949)


## Choose which HPO you want to run, default to RUN_BOTH

In [1]:
RUN_RAY = 0
RUN_OPTUNA = 1
RUN_ALL = 2

# Make your selection here
RUN = RUN_ALL

In [2]:
pip install "ray[tune]" transformers datasets scipy sklearn torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 59.1 MB 1.4 MB/s 
     |████████████████████████████████| 5.5 MB 67.3 MB/s 
     |████████████████████████████████| 451 kB 69.1 MB/s 
     |████████████████████████████████| 182 kB 91.7 MB/s 
     |████████████████████████████████| 7.6 MB 78.6 MB/s 
     |████████████████████████████████| 212 kB 94.8 MB/s 
     |████████████████████████████████| 115 kB 91.6 MB/s 
     |████████████████████████████████| 127 kB 94.5 MB/s 
     |████████████████████████████████| 8.8 MB 67.4 MB/s 
     |████████████████████████████████| 125 kB 96.5 MB/s 
     |████████████████████████████████| 468 kB 77.8 MB/s 
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=05ada864833551db8d5bc9fb12794f66d01cfc3e5b4e3ae2780a9c716e7cc329
  Stored in directory: /root/.cache/pip/wheels/42/56/cc/4a8bf86613aafd5b7f1b310477667c1fca5c51c3ae4124a003
S

In [3]:
from datasets import load_dataset, load_metric
from transformers import (AutoModelForSequenceClassification, AutoTokenizer,
                          Trainer, TrainingArguments)

In [4]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
dataset = load_dataset('glue', 'mrpc')
metric = load_metric('glue', 'mrpc')

def encode(examples):
    outputs = tokenizer(
        examples['sentence1'], examples['sentence2'], truncation=True)
    return outputs

encoded_dataset = dataset.map(encode, batched=True)

def model_init():
    return AutoModelForSequenceClassification.from_pretrained(
        'distilbert-base-uncased', return_dict=True)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Evaluate during training and a bit more often
# than the default to be able to prune bad trials early.
# Disabling tqdm is a matter of preference.
training_args = TrainingArguments(
    "test", 
    evaluation_strategy="steps", 
    eval_steps=500, 
    disable_tqdm=False)

trainer = Trainer(
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    model_init=model_init,
    compute_metrics=compute_metrics,
)





Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

...

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.24.0",
  "vocab_size": 30522
}



Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/pytorch_model.bin
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBert

##Using Ray to customize the hyper parameters

In [5]:
from typing import Dict

def hp_space_ray(trial) -> Dict[str, float]:
  from ray import tune
  return {
      "learning_rate": tune.loguniform(1e-6, 1e-4),
      "num_train_epochs": tune.choice(list(range(1, 6))),
      "seed": tune.uniform(1, 40),
      "per_device_train_batch_size": tune.choice([4, 8, 16, 32, 64]),
      "weight_decay": tune.choice([0.1, 0.3, 0.03])
  }


* un-comment this block to run Ray HPO

In [6]:
# Default objective is the sum of all metrics
# when metrics are provided, so we have to maximize it.
%%time
if RUN == RUN_RAY or RUN == RUN_ALL:
  best_run = trainer.hyperparameter_search(
    hp_space = hp_space_ray,
    direction="maximize", 
    backend="ray", 
    n_trials=3)
  print("------------RAY------------")
  print(best_run)

No `resources_per_trial` arg was passed into `hyperparameter_search`. Setting it to a default value of 1 CPU and 1 GPU for each trial.
2022-11-29 17:25:34,506	INFO worker.py:1528 -- Started a local Ray instance.
/usr/local/lib/python3.7/dist-packages/ray/tune/trainable/function_trainable.py:611: DeprecationWarning: `checkpoint_dir` in `func(config, checkpoint_dir)` is being deprecated. To save and load checkpoint in trainable functions, please use the `ray.air.session` API:

from ray.air import session

def train(config):
    # ...
    session.report({"metric": metric}, checkpoint=checkpoint)

For more information please see https://docs.ray.io/en/master/tune/api_docs/trainable.html

  DeprecationWarning,


== Status ==
Current time: 2022-11-29 17:25:38 (running for 00:00:00.19)
Memory usage on this node: 3.4/51.0 GiB 
Using FIFO scheduling algorithm.
Resources requested: 1.0/8 CPUs, 1.0/1 GPUs, 0.0/30.18 GiB heap, 0.0/15.09 GiB objects (0.0/1.0 accelerator_type:T4)
Result logdir: /root/ray_results/_objective_2022-11-29_17-25-37
Number of trials: 3/3 (2 PENDING, 1 RUNNING)
+------------------------+----------+----------------+-----------------+--------------------+------------------------+---------+----------------+
| Trial name             | status   | loc            |   learning_rate |   num_train_epochs |   per_device_train_bat |    seed |   weight_decay |
|                        |          |                |                 |                    |                ch_size |         |                |
|------------------------+----------+----------------+-----------------+--------------------+------------------------+---------+----------------|
| _objective_d6e35_00000 | RUNNING  | 172.2

(_objective pid=659) Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_projector.bias']
(_objective pid=659) - This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(_objective pid=659) - This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(_objective pid=659) Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased

== Status ==
Current time: 2022-11-29 17:25:47 (running for 00:00:09.92)
Memory usage on this node: 4.4/51.0 GiB 
Using FIFO scheduling algorithm.
Resources requested: 1.0/8 CPUs, 1.0/1 GPUs, 0.0/30.18 GiB heap, 0.0/15.09 GiB objects (0.0/1.0 accelerator_type:T4)
Result logdir: /root/ray_results/_objective_2022-11-29_17-25-37
Number of trials: 3/3 (2 PENDING, 1 RUNNING)
+------------------------+----------+----------------+-----------------+--------------------+------------------------+---------+----------------+
| Trial name             | status   | loc            |   learning_rate |   num_train_epochs |   per_device_train_bat |    seed |   weight_decay |
|                        |          |                |                 |                    |                ch_size |         |                |
|------------------------+----------+----------------+-----------------+--------------------+------------------------+---------+----------------|
| _objective_d6e35_00000 | RUNNING  | 172.2

  2%|▏         | 7/290 [00:05<02:13,  2.12it/s]


...

== Status ==
Current time: 2022-11-29 17:31:18 (running for 00:05:39.97)
Memory usage on this node: 4.4/51.0 GiB 
Using FIFO scheduling algorithm.
Resources requested: 1.0/8 CPUs, 1.0/1 GPUs, 0.0/30.18 GiB heap, 0.0/15.09 GiB objects (0.0/1.0 accelerator_type:T4)
Result logdir: /root/ray_results/_objective_2022-11-29_17-25-37
Number of trials: 3/3 (1 RUNNING, 2 TERMINATED)
+------------------------+------------+-----------------+-----------------+--------------------+------------------------+---------+----------------+-------------+
| Trial name             | status     | loc             |   learning_rate |   num_train_epochs |   per_device_train_bat |    seed |   weight_decay |   objective |
|                        |            |                 |                 |                    |                ch_size |         |                |             |
|------------------------+------------+-----------------+-----------------+--------------------+------------------------+---------+----

 95%|█████████▌| 1315/1377 [01:44<00:04, 13.52it/s]


== Status ==
Current time: 2022-11-29 17:31:23 (running for 00:05:44.97)
Memory usage on this node: 4.4/51.0 GiB 
Using FIFO scheduling algorithm.
Resources requested: 1.0/8 CPUs, 1.0/1 GPUs, 0.0/30.18 GiB heap, 0.0/15.09 GiB objects (0.0/1.0 accelerator_type:T4)
Result logdir: /root/ray_results/_objective_2022-11-29_17-25-37
Number of trials: 3/3 (1 RUNNING, 2 TERMINATED)
+------------------------+------------+-----------------+-----------------+--------------------+------------------------+---------+----------------+-------------+
| Trial name             | status     | loc             |   learning_rate |   num_train_epochs |   per_device_train_bat |    seed |   weight_decay |   objective |
|                        |            |                 |                 |                    |                ch_size |         |                |             |
|------------------------+------------+-----------------+-----------------+--------------------+------------------------+---------+----

100%|██████████| 1377/1377 [01:49<00:00, 12.58it/s]
2022-11-29 17:31:27,817	INFO tune.py:778 -- Total run time: 350.36 seconds (349.63 seconds for the tuning loop).


== Status ==
Current time: 2022-11-29 17:31:27 (running for 00:05:49.63)
Memory usage on this node: 4.4/51.0 GiB 
Using FIFO scheduling algorithm.
Resources requested: 0/8 CPUs, 0/1 GPUs, 0.0/30.18 GiB heap, 0.0/15.09 GiB objects (0.0/1.0 accelerator_type:T4)
Result logdir: /root/ray_results/_objective_2022-11-29_17-25-37
Number of trials: 3/3 (3 TERMINATED)
+------------------------+------------+-----------------+-----------------+--------------------+------------------------+---------+----------------+-------------+
| Trial name             | status     | loc             |   learning_rate |   num_train_epochs |   per_device_train_bat |    seed |   weight_decay |   objective |
|                        |            |                 |                 |                    |                ch_size |         |                |             |
|------------------------+------------+-----------------+-----------------+--------------------+------------------------+---------+----------------+--

In [7]:
print(best_run)

BestRun(run_id='d6e35_00002', objective=1.7371545003543587, hyperparameters={'learning_rate': 1.5930522616241033e-05, 'num_train_epochs': 3, 'seed': 1.8027952775362954, 'per_device_train_batch_size': 8, 'weight_decay': 0.3})


* Result of running Colab Pro +, GPU/High RAM:


```

2022-11-28 02:28:06,252	INFO tune.py:778 -- Total run time: 347.71 seconds (347.41 seconds for the tuning loop).
== Status ==
Current time: 2022-11-28 02:28:06 (running for 00:05:47.41)
Memory usage on this node: 4.4/51.0 GiB 
Using FIFO scheduling algorithm.
Resources requested: 0/8 CPUs, 0/1 GPUs, 0.0/30.17 GiB heap, 0.0/15.08 GiB objects (0.0/1.0 accelerator_type:T4)
Result logdir: /root/ray_results/_objective_2022-11-28_02-22-18
Number of trials: 3/3 (3 TERMINATED)
+------------------------+------------+-----------------+-----------------+--------------------+------------------------+---------+----------------+-------------+
| Trial name             | status     | loc             |   learning_rate |   num_train_epochs |   per_device_train_bat |    seed |   weight_decay |   objective |
|                        |            |                 |                 |                    |                ch_size |         |                |             |
|------------------------+------------+-----------------+-----------------+--------------------+------------------------+---------+----------------+-------------|
| _objective_7b66a_00000 | TERMINATED | 172.28.0.2:655  |     5.61152e-06 |                  5 |                     64 | 8.15396 |            0.1 |     1.5247  |
| _objective_7b66a_00001 | TERMINATED | 172.28.0.2:1060 |     2.05134e-06 |                  3 |                     16 | 4.89902 |            0.1 |     1.49605 |
| _objective_7b66a_00002 | TERMINATED | 172.28.0.2:1841 |     1.59305e-05 |                  3 |                      8 | 1.8028  |            0.2 |     1.72919 |
+------------------------+------------+-----------------+-----------------+--------------------+------------------------+---------+----------------+-------------+


(_objective pid=1841) {'train_runtime': 107.4607, 'train_samples_per_second': 102.4, 'train_steps_per_second': 12.814, 'train_loss': 0.3774486755060821, 'epoch': 3.0}
BestRun(run_id='7b66a_00002', objective=1.7291939932062017, hyperparameters={'learning_rate': 1.5930522616241033e-05, 'num_train_epochs': 3, 'seed': 1.8027952775362954, 'per_device_train_batch_size': 8, 'weight_decay': 0.2})
```



##HPO Using Optuna

In [8]:
from typing import Dict

def hp_space_optuna(trial) -> Dict[str, float]:
    return {
        "learning_rate" : trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
        "num_train_epochs" : trial.suggest_int("num_train_epochs", 1, 5),
        "seed" : trial.suggest_int("seed", 1, 40),
        "per_device_train_batch_size" : trial.suggest_categorical("per_device_train_batch_size", [4, 8, 16, 32, 64]),
        "weight_decay" : trial.suggest_loguniform('weight_decay', 0.001, 0.1)
    }


In [9]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 22.3 MB/s 
     |████████████████████████████████| 209 kB 86.9 MB/s 
     |████████████████████████████████| 81 kB 10.8 MB/s 
     |████████████████████████████████| 78 kB 7.9 MB/s 
     |████████████████████████████████| 112 kB 73.7 MB/s 
     |████████████████████████████████| 147 kB 84.6 MB/s 
     |████████████████████████████████| 50 kB 7.4 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=9f3ee6e81e695bc415c66bae0cb4d645ff089c0a61f7274c194a37e38bcbc88f
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [10]:
import optuna

In [11]:
%%time
if RUN == RUN_OPTUNA or RUN == RUN_ALL:
  best_run = trainer.hyperparameter_search(
      hp_space=hp_space_optuna,
      direction="maximize",
      backend="optuna",
      n_trials=3
  )
  print("------------OPTUNA------------")
  print(best_run)

[I 2022-11-29 17:31:39,523] A new study created in memory with name: no-name-d173dd73-612d-45b8-a0a6-9c5d11025337
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  if __name__ == '__main__':
Trial: {'learning_rate': 2.9378274943857116e-05, 'num_train_epochs': 5, 'seed': 27, 'per_device_train_batch_size': 64, 'weight_decay': 0.02237594008580847}
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "i

Step,Training Loss,Validation Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx. If sentence2, sentence1, idx are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 8


[I 2022-11-29 17:33:50,810] Trial 0 finished with value: 1.7224171394446104 and parameters: {'learning_rate': 2.9378274943857116e-05, 'num_train_epochs': 5, 'seed': 27, 'per_device_train_batch_size': 64, 'weight_decay': 0.02237594008580847}. Best is trial 0 with value: 1.7224171394446104.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  if __name__ == '__main__':
Trial: {'learning_rate': 6.155893160017104e-06, 'num_train_epochs': 2, 'seed': 12, 'per_device_train_batch_size': 8, 'weight_decay': 0.039499246466172394}
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base

Step,Training Loss,Validation Loss,Accuracy,F1
500,0.577700,0.496551,0.718137,0.823349


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx. If sentence2, sentence1, idx are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 8
Saving model checkpoint to test/run-1/checkpoint-500
Configuration saved in test/run-1/checkpoint-500/config.json
Model weights saved in test/run-1/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test/run-1/checkpoint-500/tokenizer_config.json
Special tokens file saved in test/run-1/checkpoint-500/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


[I 2022-11-29 17:35:03,788] Trial 1 finished with value: 1.541485949218397 and parameters: {'learning_rate': 6.155893160017104e-06, 'num_train_epochs': 2, 'seed': 12, 'per_device_train_batch_siz

Step,Training Loss,Validation Loss,Accuracy,F1
500,0.572400,0.496670,0.708333,0.820513


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx. If sentence2, sentence1, idx are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 8
Saving model checkpoint to test/run-2/checkpoint-500
Configuration saved in test/run-2/checkpoint-500/config.json
Model weights saved in test/run-2/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test/run-2/checkpoint-500/tokenizer_config.json
Special tokens file saved in test/run-2/checkpoint-500/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


[I 2022-11-29 17:36:16,840] Trial 2 finished with value: 1.528846153846154 and parameters: {'learning_rate': 7.2967030975123275e-06, 'num_train_epochs': 2, 'seed': 2, 'per_device_train_batch_siz

------------
BestRun(run_id='0', objective=1.7224171394446104, hyperparameters={'learning_rate': 2.9378274943857116e-05, 'num_train_epochs': 5, 'seed': 27, 'per_device_train_batch_size': 64, 'weight_decay': 0.02237594008580847})
CPU times: user 4min 55s, sys: 4.23 s, total: 4min 59s
Wall time: 4min 37s


* optuna finished result:
* [I 2022-11-28 08:38:32,613] Trial 2 finished with value: 1.7188552188552189 and parameters: {'learning_rate': 3.9534652778264654e-05, 'num_train_epochs': 3, 'seed': 15, 'per_device_train_batch_size': 32, 'weight_decay': 0.0023147682141046017}. 
* Best is trial 2 with value: 1.7188552188552189.
* CPU times: user 7min 53s, sys: 10.2 s, total: 8min 3s
Wall time: 8min 4s
* BestRun(run_id='2', objective=1.7188552188552189, hyperparameters={'learning_rate': 3.9534652778264654e-05, 'num_train_epochs': 3, 'seed': 15, 'per_device_train_batch_size': 32, 'weight_decay': 0.0023147682141046017}) 
* 10/28 BestRun(run_id='0', objective=1.6687864334923157, hyperparameters={'learning_rate': 6.756272182144086e-06, 'num_train_epochs': 3, 'seed': 21, 'per_device_train_batch_size': 8, 'weight_decay': 0.0017858013659287576})
* 10/29 BestRun(run_id='0', objective=1.7224171394446104, hyperparameters={'learning_rate': 2.9378274943857116e-05, 'num_train_epochs': 5, 'seed': 27, 'per_device_train_batch_size': 64, 'weight_decay': 0.02237594008580847})
* 10/29 CPU times: user 4min 55s, sys: 4.23 s, total: 4min 59s
* Wall time: 4min 37s



In [12]:
print(best_run)

BestRun(run_id='0', objective=1.7224171394446104, hyperparameters={'learning_rate': 2.9378274943857116e-05, 'num_train_epochs': 5, 'seed': 27, 'per_device_train_batch_size': 64, 'weight_decay': 0.02237594008580847})
